# A RAG application using ChatGPT API

In [ ]:
%env OPENAI_API_KEY =
MODEL = "gpt-3.5-turbo"


env: OPENAI_API_KEY=sk-proj-ktWXiVIzs0gH2bn9jouET3BlbkFJcvq4Vq4SBu85axaVGz0g


In [ ]:
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Tell me a joke")

"Why don't scientists trust atoms? \n\nBecause they make up everything!"

In [ ]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [ ]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

'My name is Santiago.'

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/Pandora Papers.pdf")
pages = loader.load_and_split()
pages

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [ ]:
retriever = vectorstore.as_retriever()
retriever.invoke("money laundering")

In [ ]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [ ]:
questions = [
    "What are the Pandora Papers and how were they obtained?",
    "Who are some of the most notable individuals mentioned in the Pandora Papers and what accusations are made against them?",
    "How do the Pandora Papers differ from previous leaks like the Panama Papers and Paradise Papers?",
    "What methods do wealthy individuals and corporations use to hide their wealth as revealed by the Pandora Papers?",
    "What are the legal and ethical implications of the findings in the Pandora Papers?",
    "How have governments around the world reacted to the revelations in the Pandora Papers?",
    "What role do offshore financial centers play in the global economy, according to insights from the Pandora Papers?",
    "How have the Pandora Papers influenced public opinion on wealth inequality and tax justice?",
    "What measures are being proposed or implemented to prevent the kind of secretive financial activities revealed by the Pandora Papers?",
    "What challenges do journalists face when investigating and reporting on leaks like the Pandora Papers?",

]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What are the Pandora Papers and how were they obtained?
Answer: The Pandora Papers are 11.9 million leaked documents with 2.9 terabytes of data that were obtained by the International Consortium of Investigative Journalists (ICIJ) and published beginning on October 3, 2021.

Question: Who are some of the most notable individuals mentioned in the Pandora Papers and what accusations are made against them?
Answer: Some of the most notable individuals mentioned in the Pandora Papers include former British prime minister Tony Blair, Chilean president Sebastián Piñera, former Kenyan president Uhuru Kenyatta, Ukrainian president Volodymyr Zelenskyy, Qatari emir Tamim bin Hamad Al Thani, United Arab Emirates prime minister and Dubai ruler Mohammed bin Rashid Al Maktoum, Gabonese president Ali Bongo Ondimba, Lebanese prime minister Najib Mikati, Ecuadorian president Guillermo Lasso, and Cypriot president Nicos Anastasiades. Accusations made against them include hiding assets in tax ha